In [ ]:
!pip install soundfile

In [ ]:
import numpy as np
import os 
import sys
import glob
import librosa
#import soundfile
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [ ]:
categories=['spoof','bonafide']

In [ ]:
def extract_features(parent_dir,sub_dirs,file_ext="*.flac",
                     bands=60,frames=41):
    def _windows(data, window_size):
        start = 0
        while start < len(data):
            yield int(start), int(start + window_size)
            start += (window_size // 2)

    window_size = 512 * (frames - 1)
    features, labels = [], []
    for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
        segment_log_specgrams, segment_labels = [], []
        sound_clip,sr = librosa.load(fn)
        categories=['spoof','bonafide']
        label = categories.index(sub_dirs)
        #label = int(fn.split('/')[2].split('-')[1])
        for (start,end) in _windows(sound_clip,window_size):
            if(len(sound_clip[start:end]) == window_size):
                signal = sound_clip[start:end]
                melspec = librosa.feature.melspectrogram(signal,n_mels=bands)
                logspec = librosa.amplitude_to_db(melspec)
                logspec = logspec.T.flatten()[:, np.newaxis].T
                segment_log_specgrams.append(logspec)
                segment_labels.append(label)

        segment_log_specgrams = np.asarray(segment_log_specgrams).reshape(
            len(segment_log_specgrams),bands,frames,1)
        segment_features = np.concatenate((segment_log_specgrams, np.zeros(
            np.shape(segment_log_specgrams))), axis=3)
        for i in range(len(segment_features)):
            segment_features[i, :, :, 1] = librosa.feature.delta(
                segment_features[i, :, :, 0])

        if len(segment_features) > 0: # check for empty segments
            features.append(segment_features)
            labels.append(segment_labels)
    return features, labels

In [ ]:
parent_dir = '/content/drive/My Drive/dataset/Samsung_PRISM/'
save_dir = "/content/drive/My Drive/dataset/Samsung_PRISM/data/"
folds = sub_dirs = np.array(['bonafide','spoof'])

In [ ]:
for sub_dir in sub_dirs:
    features, labels = extract_features(parent_dir,sub_dir)
    np.savez("{0}{1}".format(save_dir, sub_dir),
             features=features,
             labels=labels)

In [ ]:
features[0]

In [ ]:
np.shape(labels)

(199,)

In [ ]:
def get_network():
    num_filters = [24,32,64,128]
    pool_size = (2, 2)
    kernel_size = (3, 3)
    input_shape = (60, 41, 2)
    num_classes = 2
    keras.backend.clear_session()

    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(24, kernel_size,
                padding="same", input_shape=input_shape))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))

    model.add(keras.layers.Conv2D(32, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))

    model.add(keras.layers.Conv2D(64, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))

    model.add(keras.layers.Conv2D(128, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))

    model.add(keras.layers.GlobalMaxPooling2D())
    model.add(keras.layers.Dense(128, activation="relu"))
    model.add(keras.layers.Dense(num_classes, activation="sigmoid"))

    model.compile(optimizer=keras.optimizers.Adam(1e-4),
        loss='binary_crossentropy',
        metrics=["accuracy"])
    return model

In [ ]:
accuracies = []
folds = np.array(['bonafide','spoof'])
load_dir = '/content/data/'
kf = KFold(n_splits=2)
for train_index, test_index in kf.split(folds):
    x_train, y_train = [], []
    for ind in train_index:
        # read features or segments of an audio file
        train_data = np.load("{0}/{1}.npz".format(load_dir,folds[ind]),
                       allow_pickle=True)
        # for training stack all the segments so that they are treated as an example/instance
        features = np.concatenate(train_data["features"], axis=0)
        labels = np.concatenate(train_data["labels"], axis=0)
        x_train.append(features)
        y_train.append(labels)
    # stack x,y pairs of all training folds
    x_train = np.concatenate(x_train, axis = 0).astype(np.float32)
    y_train = np.concatenate(y_train, axis = 0).astype(np.float32)

    # for testing we will make predictions on each segment and average them to
    # produce signle label for an entire sound clip.
    test_data = np.load("{0}/{1}.npz".format(load_dir,
                   folds[test_index][0]), allow_pickle=True)
    x_test = test_data["features"]
    y_test = test_data["labels"]

    model = get_network()
    model.fit(x_train, y_train, epochs = 50, batch_size = 24, verbose = 0)

    # evaluate on test set/fold
    y_true, y_pred = [], []
    for x, y in zip(x_test, y_test):
        # average predictions over segments of a sound clip
        avg_p = np.argmax(np.mean(model.predict(x), axis = 0))
        y_pred.append(avg_p)
        # pick single label via np.unique for a sound clip
        y_true.append(np.unique(y)[0])
    accuracies.append(accuracy_score(y_true, y_pred))
print("Average 10 Folds Accuracy: {0}".format(np.mean(accuracies)))

Average 10 Folds Accuracy: 0.0


In [ ]:
pdata = np.load('/content/drive/My Drive/dataset/Samsung_PRISM/data/bonafide.npz',allow_pickle=True)
ndata = np.load('/content/drive/My Drive/dataset/Samsung_PRISM/data/spoof.npz',allow_pickle=True)

In [ ]:
x=[]
y=[]
features = np.concatenate(pdata["features"], axis=0)
labels = np.concatenate(pdata["labels"], axis=0)
x.extend(features)
y.extend(labels)
features = np.concatenate(ndata["features"], axis=0)
labels = np.concatenate(ndata["labels"], axis=0)
x.extend(features)
y.extend(labels)

In [ ]:
np.unique(y,return_counts=True)

(array([0, 1]), array([ 942, 1378]))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y)

In [ ]:
type(X_train)

list

In [ ]:
model=get_network()

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 41, 24)        456       
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 41, 24)        96        
_________________________________________________________________
activation (Activation)      (None, 60, 41, 24)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 20, 24)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 20, 32)        6944      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 20, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 20, 32)        0

In [ ]:
model.fit(np.array(X_train),np.array(y_train),epochs=25,batch_size=32,verbose=1,validation_data=(np.array(X_test),np.array(y_test)))

Epoch 1/25
55/55 [==============================] - 8s 150ms/step - loss: 0.4713 - accuracy: 0.7649 - val_loss: 0.8944 - val_accuracy: 0.5897
Epoch 2/25
55/55 [==============================] - 8s 146ms/step - loss: 0.2890 - accuracy: 0.8776 - val_loss: 0.6157 - val_accuracy: 0.6172
Epoch 3/25
55/55 [==============================] - 8s 144ms/step - loss: 0.2062 - accuracy: 0.9236 - val_loss: 0.2690 - val_accuracy: 0.9224
Epoch 4/25
55/55 [==============================] - 8s 145ms/step - loss: 0.1544 - accuracy: 0.9517 - val_loss: 0.2097 - val_accuracy: 0.9259
Epoch 5/25
55/55 [==============================] - 8s 151ms/step - loss: 0.1206 - accuracy: 0.9661 - val_loss: 0.1756 - val_accuracy: 0.9362
Epoch 6/25
55/55 [==============================] - 8s 143ms/step - loss: 0.0901 - accuracy: 0.9787 - val_loss: 0.1839 - val_accuracy: 0.9190
Epoch 7/25
55/55 [==============================] - 8s 143ms/step - loss: 0.0716 - accuracy: 0.9833 - val_loss: 0.2035 - val_accuracy: 0.9086
Epoch 

In [ ]:
model.fit(x, y, epochs = 3, batch_size = 24, verbose = 1)


ValueError: ignored

In [ ]:
np.shape(x_train)

(1378, 60, 41, 2)

In [ ]:
np.shape(y_test)

(199,)

In [ ]:
np.unique(y_train)

array([1.], dtype=float32)

In [ ]:
np.unique(y_test)

array([list([0]), list([0, 0]), list([0, 0, 0]), list([0, 0, 0, 0]),
       list([0, 0, 0, 0, 0]), list([0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0]), list([0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
      dtype=object)

In [ ]:
train_index

array([0])

In [ ]:
ind

0

In [ ]:
kf.split(folds)

<generator object _BaseKFold.split at 0x7f238fc29888>

In [ ]:
y_train

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)